Write a Python program based on the Week 9 class exercises that:
Implements a keyword or sentence taxonomy of topic classes.
Notes: 
1. This taxonomy can be derived from the results of your LDA topic clustering as well as  from some manual enrichment
Do not use the taxonomy provided in Class Exercises, as it may not generalize over your specific dataset
A keyword-based taxonomy will work better with Word2Vec while a sentence-based taxonomy will work better with BERT.

2. Classifies your Webhose article titles against the developed taxonomy using semantic_text_similarity package (as shown in the Class Exercise with BERT) or word2vec similarity (as shown in the Class Exercise with Word2Vec), and groups them by the topic classes

Your final submission should include:
Jupyter Notebook or Python (.py) with your implementation
Explicit JSON representation of your taxonomy 
Output showing each topic in the taxonomy and top 10 matching articles

Taxonomy (60 pts)
Implementation of keyword- or sentence-based taxonomy of topics representative of your dataset titles.

Classification (40 pts)
Classify article titles using semantic similarity of title text vs topic keywords or descriptions, and organize by topic classes.

# import data


In [1]:
import pandas as pd
df = pd.read_json ('Amazon_new.json', lines = True)

In [ ]:
df.head()

,thread,uuid,url,ord_in_thread,parent_url,author,published,title,text,highlightText,highlightTitle,highlightThreadTitle,language,external_links,external_images,entities,rating,crawled,updated,id
0,{'uuid': '5f6609ae42566bea91daa11888a1289e922c...,5f6609ae42566bea91daa11888a1289e922c412e,https://robbreport.com/lifestyle/product-recom...,0,NaN,Bryan Hood,2020-06-13T12:00:00.000+03:00,The Best Electric Coffee Grinders for Brewing ...,Four electric coffee grinders to make fresh co...,,,,english,[http://amazon.com/exec/obidos/ASIN/B01JP0LAFE...,[],{'persons': [{'name': 'de’longhi dedica burr g...,NaN,2020-06-13T19:12:27.004+03:00,2020-06-13T21:51:40.015+03:00,0
1,{'uuid': 'b7817e6a93d8c8b4ef4a0870ec78d25bdfe7...,b7817e6a93d8c8b4ef4a0870ec78d25bdfe780e5,https://scvincent.com/2020/06/13/through-the-s...,0,NaN,Sue Vincent,2020-06-13T12:00:00.000+03:00,Through the Secret Door ~ Dr. Crystal Grimes #...,Related About Sue Vincent Sue Vincent is a Yor...,,,,english,[https://goodreads.com/author/show/6551588.Sue...,[],"{'persons': [{'name': 'crystal grimes', 'senti...",NaN,2020-06-13T19:26:52.022+03:00,2020-06-13T19:26:52.022+03:00,1
2,{'uuid': '0177c898f2606497ddd4d80806016c258c90...,0177c898f2606497ddd4d80806016c258c90fca3,https://www.marketresearchreporting.com/2020/0...,0,NaN,admin,2020-06-13T11:57:00.000+03:00,New York Legal professional Normal is Intervie...,The New York legal professional basic’s workpl...,,,,english,[],[],"{'persons': [{'name': 'letitia james', 'sentim...",NaN,2020-06-13T15:43:04.005+03:00,2020-06-13T17:19:34.017+03:00,2
3,{'uuid': 'eeaa4ca8ac1103cfebb29684a9a136171dcf...,eeaa4ca8ac1103cfebb29684a9a136171dcf9fe8,https://kafkadesk.org/2020/06/13/czech-republi...,0,NaN,Kafkadesk,2020-06-13T11:56:00.000+03:00,Czech Republic agrees to lower “GAFA tax” on d...,"Comments 0 Prague, Czech Republic – The Czech ...",,,,english,"[https://addthis.com/help/api-spec, https://ww...",[],"{'persons': [{'name': 'jana maláčová', 'sentim...",NaN,2020-06-13T13:05:52.031+03:00,2020-06-13T13:05:52.031+03:00,3
4,{'uuid': '6ad23468aa10d0184c77752da9ae9051a5ad...,6ad23468aa10d0184c77752da9ae9051a5ad5e4d,https://gulfnews.com/entertainment/bollywood/a...,0,NaN,ANI,2020-06-13T11:41:00.000+03:00,Abhishek Bachchan’s debut web series ‘Breathe ...,Actor Abhishek Bachchan’s debut web series ‘Br...,,,,english,[],[],"{'persons': [{'name': 'abhishek bachchan', 'se...",NaN,2020-06-13T18:58:16.038+03:00,2020-06-13T18:58:16.038+03:00,4


# Build taxonomy

In [25]:
topic_taxonomy = {
    "Stream":
    {
        "Fire TV": "Amazon Fire TV (stylized as amazon fireTV) is a line of digital media player and microconsoles developed by Amazon.The devices are small network appliances that deliver digital audio and video content streamed via the Internet to a connected high-definition television. ",
        "Prime Video": "Prime Video is an American Internet video on demand service that is developed, owned, and operated by Amazon.It offers television shows and films for rent or purchase and Prime Video, a selection of Amazon Studios original content and licensed acquisitions included in the Amazon's Prime subscription. ",
        "Amazon Music" : "Amazon Music (previously Amazon MP3) is a music streaming platform and online music store operated by Amazon.",
        "Amazon Alexa": "Alexa is a virtual assistant AI technology developed by Amazon. It is capable of voice interaction, music playback, making to-do lists, setting alarms, streaming podcasts, playing audiobooks, and providing weather, traffic, sports, and other real-time information"
    },
    "Business":
    {
        "Employment": "Employment is a relationship between two parties, usually based on contract where work is paid for, where one party, which may be a corporation, for profit, not-for-profit organization, co-operative or other entity is the employer and the other is the employee",
        "Divestiture": "Disposition or sale of an asset by a company through sale, split or spinoff",
        "Competition": "The process of companies and individuals competing in the same industry or field",
        "Stock": "Stock (also capital stock) of a corporation, is all of the shares into which ownership of the corporation is divided."
    },
    "Service":
    {
        "Delivery": "Delivery is the process of transporting goods from a source location to a predefined destination.",
        "Prime":"Amazon Prime is a paid subscription program from Amazon that gives users access to additional services otherwise unavailable or available at a premium to regular Amazon customers.",
        "Return & Refund": "Return & Refund is the process of a customer taking previously purchased merchandise back to the retailer, and in turn receiving a refund in the original form of payment, exchange for another item (identical or different), or a store credit.",
       
    },
    "Trending":
    {
         "COVID-19": "Coronavirus disease 2019 (COVID-19) is an infectious disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2)."
    }
}

In [ ]:
topic_list = []
label_list = []
for key, value in topic_taxonomy.items():
    for label, keywords in value.items():
        topic_list.append(keywords.lower())
        label_list.append(label)

# BERT 

In [3]:
from semantic_text_similarity.models import WebBertSimilarity

model = WebBertSimilarity(device='cpu', batch_size=10) #defaults to GPU prediction

In [ ]:
BERT = pd.DataFrame(columns=['title', 'topic','subtopic','similartiy'])
n = 0
for i in range(len(df)):
  topics = {}
  for (topic,value) in topic_taxonomy.items():
    for (subtopic, description) in value.items():
      try:
        topics[(topic,subtopic)] =  model.predict([(df['title'][i], description)])[0]  
      except:
        n += 1
  sort_orders = sorted(topics.items(), key=lambda x: x[1], reverse = True)[0]
  new = [df['title'][i],sort_orders[0][0], sort_orders[0][1],sort_orders[1] ]
  new_series = pd.Series(new, index = BERT. columns)
  BERT = BERT.append(new_series, ignore_index= True)

In [27]:
BERT.sort_values(by='similartiy', ascending= False)[:20]

,title,topic,subtopic,similartiy
42,How To Get Free One Year Amazon Prime Subscrip...,Service,Prime,2.878927
99,"AT AMAZON, Shop Amazon Warehouse Deals – Deep ...",Service,Prime,2.281258
63,Amazon Prime Video app now available on Jio se...,Service,Prime,2.276286
31,Fire TV Edition and Roku HD and 4K TVs are on ...,Stream,Fire TV,2.162013
29,"Every New Movie Coming To Netflix, Amazon Prim...",Stream,Prime Video,1.814051
22,"MultiChoice (DStv, GOtv) Signs Distribution Pa...",Stream,Prime Video,1.786640
90,Amazon: Amazon's enterprise practices examined...,Service,Prime,1.726570
97,"Everything New On Netflix, Disney Plus, Amazon...",Service,Prime,1.663919
39,Sources: California is conducting an inquiry i...,Business,Competition,1.621407
5,Amazon's business practices examined by two US...,Service,Prime,1.598660
